In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
TRAIN_P='../input/train/'
TRAIN_T='../input/test/'
train_df=pd.read_csv('../input/train.csv')
print(train_df[0:5].Id)
#train_df=train_df[0:5000]
print(train_df.shape)
#we treat this like unet arquitect and like a mask we replace target to 0 to 27 outputs from 0 to 1 values
NUM_CLASSES=28
NUM_SLIDES=16
y_df=[[int(i) for i in s.split()] for s in train_df['Target']]
y_df_final=np.zeros((train_df.shape[0],NUM_CLASSES)).astype(int)

for i,x in enumerate(y_df):
    for val in x:
        #print(x,val,i)    
        y_df_final[i,val]=1
print(y_df_final[1])   
print(y_df_final.shape)

train_path=np.array(TRAIN_P+train_df['Id'])
print(train_path.shape)
print(train_path[0])

In [ ]:
#now we prepare train test split

from sklearn.model_selection import train_test_split
xtrain,xtest,y_train,y_test = train_test_split(
   train_path,y_df_final, test_size=0.06, random_state=42)

xtrain=xtrain.reshape(xtrain.shape[0],1)
xtest=xtest.reshape(xtest.shape[0],1)
print(xtrain.shape)
print(y_train.shape)
print(xtest.shape)
print(y_test.shape)

In [ ]:
#we have full path from trian folder and test folder to working with autoencodes
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img#,save_img
from skimage.transform import resize
from skimage import util 
import cv2
from PIL import Image

def load_image(path):
    R = np.array(Image.open(path+'_red.png'))
    G = np.array(Image.open(path+'_green.png'))
    B = np.array(Image.open(path+'_blue.png'))
    Y = np.array(Image.open(path+'_yellow.png'))
   
    image = np.stack((
        R/2 + Y/2, 
        G/2 + Y/2, 
        B),-1)
    
    
    
    #image = np.divide(image, 255)
    return image  
img=load_image(np.squeeze(xtrain[1]))
print(img.shape)
plt.imshow(img)

In [ ]:
from math import sqrt, ceil, floor
def validate_image(image, number_tiles):
    """Basic sanity checks prior to performing a split."""
    TILE_LIMIT = 99 * 99

    try:
        number_tiles = int(number_tiles)
    except:
        raise ValueError('number_tiles could not be cast to integer.')

    if number_tiles > TILE_LIMIT or number_tiles < 2:
        raise ValueError('Number of tiles must be between 2 and {} (you \
                          asked for {}).'.format(TILE_LIMIT, number_tiles))
        
def calc_columns_rows(n):
    """
    Calculate the number of columns and rows required to divide an image
    into ``n`` parts.
    Return a tuple of integers in the format (num_columns, num_rows)
    """
    num_columns = int(ceil(sqrt(n)))
    num_rows = int(ceil(n / float(num_columns)))
    return (num_columns, num_rows)
def xslice(imag, number_tiles):
    """
    Split an image into a specified number of tiles.
    Args:
       filename (str):  The filename of the image to split.
       number_tiles (int):  The number of tiles required.
    Kwargs:
       save (bool): Whether or not to save tiles to disk.
    Returns:
        Tuple of :class:`Tile` instances.
    """
    im = Image.fromarray(np.uint8(imag))
    validate_image(im, number_tiles)

    im_w, im_h = im.size
    columns, rows = calc_columns_rows(number_tiles)
    extras = (columns * rows) - number_tiles
    tile_w, tile_h = int(floor(im_w / columns)), int(floor(im_h / rows))

    tiles = []
    number = 1
    for pos_y in range(0, im_h - rows, tile_h): # -rows for rounding error.
        for pos_x in range(0, im_w - columns, tile_w): # as above.
            area = (pos_x, pos_y, pos_x + tile_w, pos_y + tile_h)
            image = im.crop(area)
            position = (int(floor(pos_x / tile_w)) + 1,
                        int(floor(pos_y / tile_h)) + 1)
            coords = (pos_x, pos_y)
            #tile = Tile(image, number, position, coords)
            tiles.append(np.array(image)/255)
            number += 1
    
    return tuple(tiles)
slices=xslice(img,NUM_SLIDES)

slices

In [ ]:
plt.imshow(slices[0])
plt.show()
plt.imshow(slices[1])
plt.show()
plt.imshow(slices[2])
plt.show()
plt.imshow(slices[3])
plt.show()

print(np.max(slices[3]))
print(slices[3].shape)

In [ ]:
slide_x=slices[0].shape[0]
slide_y=slices[0].shape[1]
slide_z=slices[0].shape[2]
def data_generator(paths,labels,batch_size):
    while True:        
        random_indexes = np.random.choice(len(paths), batch_size)        
        batch_images = np.empty((batch_size,NUM_SLIDES, slide_x, slide_y, slide_z))                        
        batch_labels = np.zeros((batch_size, 28))                        
        for i, idx in enumerate(random_indexes):
            #load image in rgbd mode 
        
            name=np.squeeze(paths[idx])
            img=load_image(str(name))
            slides=xslice(img,NUM_SLIDES)
            for x,slide in enumerate(slides):
                batch_images[i][x]=slide
                
            batch_labels[i]=labels[idx]
        yield batch_images,batch_labels
        
tt,tt2=next(data_generator(xtest,y_test,64))
gentrain=data_generator(xtrain,y_train,32)
gentest=data_generator(xtest,y_test,64)
print(tt.shape)
print(tt2.shape)
print("DONE")        

In [ ]:
import keras
from keras.layers import Dense, Dropout, LSTM,GRU
from keras.layers import Conv2D, Flatten,MaxPooling2D,GlobalMaxPooling2D
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed
from keras.models import Model
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
from keras.models import Model
from keras.layers import merge,Input,Dense, concatenate, Conv2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout,Convolution2D,Activation,Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from keras.utils.vis_utils import plot_model as plot
from keras.optimizers import SGD

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

def calculating_class_weights(y_true):
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_true)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_true[:, i])
    return weights

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*K.binary_crossentropy(y_true, y_pred), axis=-1)
    return weighted_loss

def get_weighted_loss_f1(weights):
    def weighted_loss_f1(y_true, y_pred):
        return K.mean((weights[:,0]**(1-y_true))*(weights[:,1]**(y_true))*f1_loss(y_true, y_pred), axis=-1)
    return weighted_loss_f1

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


weigh=calculating_class_weights(y_train)

In [ ]:
# some basic useless model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model, Model
from keras.layers import Bidirectional,TimeDistributed,GRU,LSTM,Activation, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Concatenate, ReLU, LeakyReLU, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras import metrics
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras import backend as K
import keras
import tensorflow as tf

from tensorflow import set_random_seed

def create_model(input_shape):
    
    dropRate = 0.25
    
    init = Input(input_shape)
    x = TimeDistributed(BatchNormalization(axis=-1))(init)
    x = TimeDistributed(Conv2D(32, (3, 3)))(x) #, strides=(2,2))(x)
    x = TimeDistributed(ReLU())(x)

    x = TimeDistributed(BatchNormalization(axis=-1))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
    ginp1 = TimeDistributed(Dropout(dropRate))(x)
    
    x = TimeDistributed(BatchNormalization(axis=-1))(ginp1)
    x = TimeDistributed(Conv2D(64, (3, 3), strides=(2,2)))(x)
    x = TimeDistributed(ReLU())(x)
    x = TimeDistributed(BatchNormalization(axis=-1))(x)
    x = TimeDistributed(Conv2D(64, (3, 3)))(x)
    x = TimeDistributed(ReLU())(x)
    x = TimeDistributed(BatchNormalization(axis=-1))(x)
    x = TimeDistributed(Conv2D(64, (3, 3)))(x)
    x = TimeDistributed(ReLU())(x)
    
    x = TimeDistributed(BatchNormalization(axis=-1))(x)
    x = TimeDistributed(MaxPooling2D(pool_size=(2, 2)))(x)
    ginp2 =TimeDistributed( Dropout(dropRate))(x)
    
    x = TimeDistributed(BatchNormalization(axis=-1))(ginp2)
    x = TimeDistributed(Conv2D(128, (3, 3)))(x)
    x = TimeDistributed(ReLU())(x)
    x =TimeDistributed( BatchNormalization(axis=-1))(x)
    x = TimeDistributed(Conv2D(128, (3, 3)))(x)
    x = TimeDistributed(ReLU())(x)
    x = TimeDistributed(BatchNormalization(axis=-1))(x)
    x = TimeDistributed(Conv2D(128, (3, 3)))(x)
    x = TimeDistributed(ReLU())(x)
    ginp3 = TimeDistributed(Dropout(dropRate))(x)
    
    gap1 = TimeDistributed(GlobalAveragePooling2D())(ginp1)    
    gap2 = TimeDistributed(GlobalAveragePooling2D())(ginp2)
    gap3 = TimeDistributed(GlobalAveragePooling2D())(ginp3)
    
    x = Concatenate()([gap1, gap2, gap3])
    x = BatchNormalization(axis=-1)(x)
    x = Bidirectional(LSTM(128,return_sequences=True))(x)
  #  x = Bidirectional()(x)
    x = Dropout(dropRate)(x)
    x = BatchNormalization(axis=-1)(x)
    x = Bidirectional(LSTM(128))(x)
   # x = Bidirectional()(x)
    x = Dropout(dropRate)(x)
    
    #x = BatchNormalization(axis=-1)(x)
    #x = Dense(1024, activation='relu')(x)
    #x = Dropout(dropRate)(x)
    
    #x = BatchNormalization(axis=-1)(x)
    #x = Dense(1024, activation='relu')(x)
    #x = Dropout(0.1)(x)
    
    x = Dense(28)(x)
    x = Activation('sigmoid')(x)
    
    model = Model(init, x)
    
    return model

model=create_model((NUM_SLIDES, slide_x, slide_y, slide_z))
model.summary()


In [ ]:
def get_model2(input_shape):
    init = Input(input_shape)    
    x=TimeDistributed(Conv2D(32, (3, 3)))(init)
    x=TimeDistributed((Activation('relu')))(x)
    x=TimeDistributed((MaxPooling2D(pool_size=(2, 2))))(x)

    x=TimeDistributed((Conv2D(32, (3, 3))))(x)
    x=TimeDistributed((Activation('relu')))(x)
    x=TimeDistributed((MaxPooling2D(pool_size=(2, 2))))(x)

    x=TimeDistributed((Conv2D(64, (3, 3))))(x)
    x=TimeDistributed((Activation('relu')))(x)
    x=TimeDistributed((MaxPooling2D(pool_size=(2, 2))))(x)

# the model so far outputs 3D feature maps (height, width, features)


    x=TimeDistributed((Flatten()))(x)  # this converts our 3D feature maps to 1D feature vectors
    x = Bidirectional(LSTM(128,return_sequences=True))(x)    
    x = Bidirectional(LSTM(128))(x)
    x =Dense(512)(x)
    x =Activation('relu')(x)
    x =Dense(28)(x)
    x=Activation('sigmoid')(x)
    model = Model(init, x)
    return model
#model=get_model2((NUM_SLIDES, slide_x, slide_y, slide_z))
#model.summary()
#model.compile(optimizer="rmsprop", loss=f1_loss,metrics=["acc",f1_loss,"binary_crossentropy",f1])
#model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

In [ ]:
from keras import optimizers
#model = load_model('/kaggle/working/class.model',custom_objects={'f1_loss': f1_loss,'f1':f1})
#sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#opt=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=["acc",f1_loss,"binary_crossentropy",f1])
print("DONE")

In [ ]:
from keras.callbacks import ModelCheckpoint
checkpointer = ModelCheckpoint(
    '/kaggle/working/class.model',
    verbose=2, save_best_only=True)

history=model.fit_generator(gentrain,
                    steps_per_epoch=100,epochs=100,
                    verbose=1,
                    validation_data=next(gentest),
                    validation_steps=10,
                    callbacks=[checkpointer]
                   )

In [ ]:
from keras.models import load_model
model = load_model('/kaggle/working/class.model',custom_objects={'f1_loss': f1_loss,'f1':f1})
model.summary()

In [ ]:
from sklearn.metrics import f1_score

tr,y_true=next(data_generator(xtest,y_test,450))
y_pred=model.predict(tr)
y_pred.shape

In [ ]:
for x,row in enumerate(y_pred):
    for y,rcol in enumerate(row):
        if y_pred[x][y]>0.5:
            y_pred[x][y]=1
        else:
            y_pred[x][y]=0
        
print("DONE")    


In [ ]:

print(f1_score(y_true, y_pred, average='macro')  )

print(f1_score(y_true, y_pred, average='micro')  )

print(f1_score(y_true, y_pred, average='weighted')  )

print(f1_score(y_true, y_pred, average=None))

In [ ]:
def show_history(history):
    fig, ax = plt.subplots(1, 3, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('f1_loss')
    ax[1].plot(history.epoch, history.history["f1_loss"], label="Train f1 loss")
    ax[1].plot(history.epoch, history.history["val_f1_loss"], label="Validation f1 loss")
    ax[2].set_title('f1')
    ax[2].plot(history.epoch, history.history["f1"], label="Train f1")
    ax[2].plot(history.epoch, history.history["val_f1"], label="Validation f1")
    
    ax[0].legend()
    ax[1].legend()
    ax[2].legend()
    

show_history(history)